# International Trade - Equities

## Imports

In [197]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v5 as pmp

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Global Variables

In [198]:
frequency = 1
t_cost = 0
target_vol = 0.10
min_regions = 4

## Data

### Riskfree Data

In [199]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_14191/2677495952.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Farma Factors Data

In [200]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_14191/297690613.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,-0.0184,0.0189,0.0745,0.0695
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789,-0.0132
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958,-0.1181
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048,0.0574
1980-05-31,0.0526,0.0200,0.0038,0.0043,-0.0063,-0.0118,0.0618
...,...,...,...,...,...,...,...
2025-05-31,0.0606,-0.0072,-0.0288,0.0129,0.0251,0.0221,0.0256
2025-06-30,0.0486,-0.0002,-0.0160,-0.0320,0.0145,-0.0264,0.0527
2025-07-31,0.0198,-0.0015,-0.0127,-0.0029,-0.0208,-0.0096,0.0184


### Benchmark Data

In [201]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()

benchmark_return = benchmark_data[['MSCI World']].pct_change()
benchmark_return = benchmark_return.squeeze()
benchmark_return

Date
1986-12-31         NaN
1987-01-31         NaN
1987-02-28         NaN
1987-03-31         NaN
1987-04-30         NaN
                ...   
2025-07-31    0.013121
2025-08-31    0.026408
2025-09-30    0.032574
2025-10-31    0.020226
2025-11-30    0.003149
Freq: ME, Name: MSCI World, Length: 468, dtype: float64

### Trade Data

In [202]:
# --- Load Trade Data ---
trade_sheet_map = {
    'US':'US', 
    'EU':'EU', 
    'JP':'JP', 
    'UK':'UK', 
    'AU':'AU', 
    'CH':'CH',
    'EM':'EM'
}

trade_export_matrices = {}

for country, trade_sheet in trade_sheet_map.items():
    display(f"Loading{country} from sheet: '{trade_sheet}'...")

    try:
        df = pd.read_excel(
        f"{path}/International Trade Data.xlsx",
        sheet_name = trade_sheet,
        index_col = 0,
        parse_dates = True
    )

        current_matrix = df.resample('YE').last()
        trade_export_matrices[country] = current_matrix

        print(f"   -> Successfully loaded matrix size: {current_matrix.shape}")
    except ValueError as e:
        print(f"   -> ERROR: Could not find sheet '{trade_sheet}'. Check Excel tabs for spaces.")
    except Exception as e:
        print(f"   -> An error occurred: {e}")


us_trade_weights = trade_export_matrices['US']
display(us_trade_weights)

jp_to_us = trade_export_matrices['JP']['US']
display(jp_to_us)

#     # Resample to Month Start ('MS')
#     export_weights = df.iloc[:, 0].resample('ME').last()
#     export_weights.name = country
#     trade_export_list.append(export_weights)

# # display(trade_export_list)
# trade_matrix = pd.concat(trade_export_list, axis = 1)
# display(trade_matrix)

# trade_data.index = pd.to_datetime(trade_data.index)
# trade_data.index = trade_data.index + pd.offsets.MonthEnd(0)

# start_date = trade_data.index.min()
# end_date = pd.to_datetime('2025-10-31')
# monthly_index = pd.date_range(start = start_date, end = end_date, freq ='ME')

# trade_weight = trade_data.reindex(monthly_index)

# trade_weight = trade_weight.ffill()

# display(trade_weight)

"LoadingUS from sheet: 'US'..."

   -> Successfully loaded matrix size: (46, 6)


"LoadingEU from sheet: 'EU'..."

   -> Successfully loaded matrix size: (28, 6)


"LoadingJP from sheet: 'JP'..."

...

   -> Successfully loaded matrix size: (46, 6)


"LoadingEM from sheet: 'EM'..."

   -> Successfully loaded matrix size: (28, 6)


,UK,CH,JP,AU,EU,EM
Date,,,,,,
1980-12-31,0.136825,0.023996,0.239271,0.047531,0.514998,0.037379
1981-12-31,0.123397,0.023089,0.254746,0.060800,0.490764,0.047204
1982-12-31,0.125185,0.022222,0.259506,0.062222,0.484691,0.046173
1983-12-31,0.131863,0.028077,0.285285,0.053898,0.469291,0.031587
1984-12-31,0.136969,0.021505,0.294163,0.061700,0.443676,0.041987
1985-12-31,0.138330,0.019366,0.282696,0.062626,0.440644,0.056338
1986-12-31,0.125779,0.022281,0.294921,0.057738,0.452084,0.047197
1987-12-31,0.133333,0.023877,0.281324,0.056028,0.462884,0.042553
1988-12-31,0.137274,0.022164,0.300524,0.056482,0.436368,0.047188


Date
1980-12-31         NaN
1981-12-31         NaN
1982-12-31         NaN
1983-12-31         NaN
1984-12-31         NaN
1985-12-31         NaN
1986-12-31         NaN
1987-12-31         NaN
1988-12-31         NaN
1989-12-31         NaN
1990-12-31         NaN
1991-12-31         NaN
1992-12-31         NaN
1993-12-31         NaN
1994-12-31         NaN
1995-12-31         NaN
1996-12-31         NaN
1997-12-31         NaN
1998-12-31    0.515010
1999-12-31    0.522850
2000-12-31    0.532162
2001-12-31    0.519347
2002-12-31    0.493894
2003-12-31    0.428211
2004-12-31    0.407359
2005-12-31    0.401429
2006-12-31    0.408665
2007-12-31    0.373898
2008-12-31    0.343769
2009-12-31    0.342568
2010-12-31    0.307754
2011-12-31    0.295027
2012-12-31    0.343397
2013-12-31    0.352715
2014-12-31    0.347436
2015-12-31    0.371093
2016-12-31    0.363281
2017-12-31    0.345312
2018-12-31    0.332192
2019-12-31    0.349064
2020-12-31    0.315299
2021-12-31    0.308064
2022-12-31    0.335324
2023-1

### Currency Data

In [203]:
# --- Load Currency Prices ---
currency_data = pd.read_excel(
    f"{path}/FX Data.xlsx",
    sheet_name = 'SPOT',
    index_col = 0,
    parse_dates = True
)

currency_data = currency_data.rename(columns={
    'CHFUSD' : 'CH',
    'EURUSD' : 'EU',
    'AUDUSD' : 'AU',
    'GBPUSD' : 'UK',
    'JPYUSD' : 'JP',
    'CNHUSD' : 'EM'
})

currency_data.index = pd.to_datetime(currency_data.index)
currency_data.index = currency_data.index + pd.offsets.MonthEnd(0)
currency_spot = currency_data

display(currency_spot)

,CH,EU,JP,AU,UK,EM
Date,,,,,,
1988-12-31,0.6658,NaN,0.007997,0.8555,1.8110,NaN
1989-01-31,0.6251,NaN,0.007663,0.8890,1.7520,NaN
1989-02-28,0.6431,NaN,0.007882,0.8015,1.7434,NaN
1989-03-31,0.6015,NaN,0.007532,0.8195,1.6855,NaN
1989-04-30,0.5976,NaN,0.007527,0.7942,1.6900,NaN
...,...,...,...,...,...,...
2025-06-30,1.2609,1.1787,0.006943,0.6581,1.3732,0.1397
2025-07-31,1.2311,1.1415,0.006634,0.6425,1.3207,0.1387
2025-08-31,1.2492,1.1686,0.006800,0.6540,1.3504,0.1404


### Equity Data

In [204]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    f"{path}/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

display(equity_prices)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,1206.821289,NaN,NaN,7.33429,7487.55371,NaN,NaN
1997-10-31,1168.258667,NaN,NaN,6.82642,7084.46436,NaN,NaN
1997-11-30,1207.453491,NaN,NaN,6.34016,7152.55615,NaN,NaN
1997-12-31,1223.840210,NaN,NaN,5.86087,7365.37451,NaN,NaN
1998-01-31,1234.778442,NaN,NaN,6.36481,7690.75049,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,6308.888672,5641.31592,14994.83984,19.64406,12121.12598,14955.800000,6276.48438
2025-07-31,6430.451172,5627.64307,14552.98438,19.41757,12131.93066,15094.600000,6109.94238
2025-08-31,6529.819336,5881.38037,15274.74121,20.75129,12521.77637,15223.700000,6284.96094


In [205]:
equity_XR =  equity_prices.pct_change()
display(equity_XR)

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,-0.031954,NaN,NaN,-0.069246,-0.053835,NaN,NaN
1997-11-30,0.033550,NaN,NaN,-0.071232,0.009611,NaN,NaN
1997-12-31,0.013571,NaN,NaN,-0.075596,0.029754,NaN,NaN
1998-01-31,0.008938,NaN,NaN,0.085984,0.044176,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.047850,0.034649,0.008742,0.018397,0.017408,0.087457,0.024923
2025-07-31,0.019268,-0.002424,-0.029467,-0.011530,0.000891,0.009281,-0.026534
2025-08-31,0.015453,0.045088,0.049595,0.068686,0.032134,0.008553,0.028645


## Signal Generation

In [206]:
# --- Compute International Trade Signal ---
# 1. INVERT QUOTES where necessary (Make everything USD base)
# Example: If you have EURUSD, invert it to get USDEUR
# (Skip this if your data is already "Foreign per USD")

currency_spot_US_base = currency_spot.copy()
currency_spot_US_base['EU'] = 1 / currency_spot['EU'] 
currency_spot_US_base['CH'] = 1 / currency_spot['CH'] 
currency_spot_US_base['JP'] = 1 / currency_spot['JP'] 
currency_spot_US_base['AU'] = 1 / currency_spot['AU']
currency_spot_US_base['UK'] = 1 / currency_spot['UK'] 
currency_spot_US_base['EM'] = 1 / currency_spot['EM']
currency_spot_US_base['USD'] = 1.0

display('Base Currency: US (USD)')
display(currency_spot_US_base)

'Base Currency: US (USD)'

,CH,EU,JP,AU,UK,EM,USD
Date,,,,,,,
1988-12-31,1.501953,NaN,125.046893,1.168907,0.552181,NaN,1.0
1989-01-31,1.599744,NaN,130.497194,1.124859,0.570776,NaN,1.0
1989-02-28,1.554968,NaN,126.871352,1.247661,0.573592,NaN,1.0
1989-03-31,1.662510,NaN,132.766861,1.220256,0.593296,NaN,1.0
1989-04-30,1.673360,NaN,132.855055,1.259129,0.591716,NaN,1.0
...,...,...,...,...,...,...,...
2025-06-30,0.793084,0.848392,144.029958,1.519526,0.728226,7.158196,1.0
2025-07-31,0.812282,0.876040,150.738619,1.556420,0.757174,7.209805,1.0
2025-08-31,0.800512,0.855725,147.058824,1.529052,0.740521,7.122507,1.0


In [207]:
cross_rates_dict = {}

for new_base_currency in currency_spot_US_base.columns:
    # Formula: Divide the Whole DataFrame by the New Base Currency column
    # axis=0 aligns the division by index (dates)
    cross_rates_dict[new_base_currency] = currency_spot_US_base.div(currency_spot_US_base[new_base_currency], axis = 0).dropna()

display("Base Currency: EU (EUR)")
display(cross_rates_dict['EU'])

display("Base Currency: JP (JPY)")
display(cross_rates_dict['JP'])

'Base Currency: EU (EUR)'

,CH,EU,JP,AU,UK,EM,USD
Date,,,,,,,
2010-08-31,1.287179,1.0,106.751978,1.423759,0.826166,8.590786,1.2680
2010-09-30,1.339688,1.0,113.872881,1.409782,0.867524,9.029139,1.3634
2010-10-31,1.370039,1.0,112.132176,1.418099,0.869622,9.181698,1.3947
2010-11-30,1.302729,1.0,108.644351,1.354088,0.834276,8.638057,1.2983
2010-12-31,1.251660,1.0,108.609916,1.307925,0.857289,8.811060,1.3384
...,...,...,...,...,...,...,...
2025-06-30,0.934808,1.0,169.768112,1.791065,0.858360,8.437366,1.1787
2025-07-31,0.927220,1.0,172.068134,1.776654,0.864314,8.229993,1.1415
2025-08-31,0.935479,1.0,171.852941,1.786850,0.865373,8.323362,1.1686


'Base Currency: JP (JPY)'

,CH,EU,JP,AU,UK,EM,USD
Date,,,,,,,
2010-08-31,0.012058,0.009368,1.0,0.013337,0.007739,0.080474,0.011878
2010-09-30,0.011765,0.008782,1.0,0.012380,0.007618,0.079291,0.011973
2010-10-31,0.012218,0.008918,1.0,0.012647,0.007755,0.081883,0.012438
2010-11-30,0.011991,0.009204,1.0,0.012463,0.007679,0.079508,0.011950
2010-12-31,0.011524,0.009207,1.0,0.012042,0.007893,0.081126,0.012323
...,...,...,...,...,...,...,...
2025-06-30,0.005506,0.005890,1.0,0.010550,0.005056,0.049699,0.006943
2025-07-31,0.005389,0.005812,1.0,0.010325,0.005023,0.047830,0.006634
2025-08-31,0.005443,0.005819,1.0,0.010398,0.005036,0.048433,0.006800


In [ ]:
# # 2. CALCULATE 1-YEAR RETURN (Pct Change)
currency_spot_US_base_chg = currency_spot_US_base.pct_change(12)

display('YoY% Inverted Spot')
display(currency_spot_US_base_chg)

# # 3. CALCULATE WEIGHTED BASKET CHANGE
# # This creates a single column representing the USD Index return
usd_basket_return = (
    (currency_spot_US_base_chg['EU'] * trade_weight['EU']) +
    (currency_spot_US_base_chg['CH'] * trade_weight['CH']) +
    (currency_spot_US_base_chg['JP'] * trade_weight['JP']) +
    (currency_spot_US_base_chg['AU'] * trade_weight['AU']) +
    (currency_spot_US_base_chg['UK'] * trade_weight['UK']) +
    (currency_spot_US_base_chg['EM'] * trade_weight['EM'])
)

display(usd_basket_return)

# # 4. CREATE THE SIGNAL (Depreciation)
# We want Positive values to mean Depreciation (Weak Dollar)
# because Weak Dollar is usually the "Buy" signal for this strategy.
fx_signal = -1 * usd_basket_return

display(fx_signal)

'YoY% Inverted Spot'

,CH,EU,JP,AU,UK,EM,USD
Date,,,,,,,
1988-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,-0.117535,-0.091117,-0.104710,0.013524,-0.079158,-0.019327,0.0
2025-07-31,-0.074811,-0.051599,0.004974,0.018210,-0.026577,-0.002163,0.0
2025-08-31,-0.057717,-0.054595,0.006029,0.034404,-0.027918,0.004274,0.0


NameError: name 'trade_weight' is not defined